In [1]:
#If needed
#!pip install tensorflow -q
#!pip install opencv-python -q
#!pip install tensorflow-addons -q

In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import json

from modules.pym_dataframe import get_clean_dataframe
from modules.pym_image_preprocessing import preprocess
from modules.pym_crnn import ctc_loss, build_crnn
from modules.pym_encoding import encode_labels

In [4]:
# Prepare train & test data
batch_size = 64
imgSize = (32, 128) 

df = get_clean_dataframe()
X_train_path, X_test_path, y_train, y_test = train_test_split(df.path, df.transcription_word, train_size=0.9, random_state=1234)

dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))
dataset_train = dataset_train.shuffle(10000).map(
    lambda x, y : [preprocess(x, imgSize, scale=0.8), y]).batch(batch_size, drop_remainder=True)
X_t, y_t = next(iter(dataset_train))

dataset = dataset_train.map(lambda X,y : [X, encode_labels(y)])

dataset_test = tf.data.Dataset.from_tensor_slices((X_test_path, y_test))
dataset_test = dataset_test.map(
    lambda x, y : [preprocess(x, imgSize, scale=1), y]).batch(batch_size, drop_remainder=True)
dataset_test = dataset_test.map(lambda X,y : [X, encode_labels(y)])

In [ ]:
# For the first run we call the code below to build the CRNN
#model = build_crnn(input_shape = X_t.shape[1:])
# For the next run we load the model from the previous epoc
model = tf.keras.models.load_model('./models/cnn_rnn_20.h5', compile=False)
# model.summary()

C:\Users\ybeil\anaconda3\lib\site-packages\keras\layers\core\lambda_layer.py:303: UserWarning: modules.pym_models is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


In [ ]:
# We could have used some callback or for loops to run the training on 20 epoc
# But : a train last 20 min and take 100% CPU => For 20 epoch, it's 6h where my personal computer is at 100% CPU and not avalaible to do something else
# So instead, i ran each training one by one (during breaks and free time), saved the model and the scores after each epoch 
# and adapt the learning rate according to the loss evolution
model.compile(loss=ctc_loss, optimizer=Adam(1e-5))
history = model.fit(dataset, epochs=1, validation_data=dataset_test)
model.save('./models/cnn_rnn_20.h5') # save the model 
json.dump(history.history, open("./models/cnn_rnn_20.json", 'w')) # save the loss and val_loss